In [1]:
# need this package for the pipeline script
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 3.0 MB/s eta 0:00:00


In [2]:
import cuml
from cuml.ensemble import RandomForestRegressor as cuRF_Regressor
from cuml.ensemble import RandomForestClassifier as cuRF_Classifier
# Install with desired log level before other imports
cuml.accel.install(log_level="debug")

[cuml.accel] Enabled managed memory.
[cuml.accel] Accelerator installed.


In [3]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import os
import sys
import os
from google.colab import drive

# force remount
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)



Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [12]:
!ls "/content/drive/My Drive/CIS 5200 Final Project/code"


 eda					  model_train
 flight_delay_pipeline_log_transform.py   pipeline_lgbm.py
 flight_delay_pipeline.py		  __pycache__
 flight_delay_pipeline_simple.py	  rf_classification.py
 HurdleModel.py				  rf_pipe_regression.py
 LGBMRegr_pipeline.py			  rf_regressor.py
'Model Evaluation.ipynb'		  setup.py


In [4]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python312.zip',
 '/usr/lib/python3.12',
 '/usr/lib/python3.12/lib-dynload',
 '',
 '/usr/local/lib/python3.12/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.12/dist-packages/IPython/extensions',
 '/root/.ipython']

In [5]:
import sys

# 2. Add your project path to Python's system path
# This tells Python: "Look in this folder when I try to import things"
project_path = '/content/drive/My Drive/CIS 5200 Final Project/code'

if project_path not in sys.path:
    sys.path.append(project_path)

# can use importlib to safely reload if you change the script
import rf_regressor
import importlib
importlib.reload(rf_regressor)
from rf_regressor import train_regressor, InferencePipeline, get_regressor_pipeline


print("Successfully loaded pipeline!")



Successfully loaded pipeline!


In [ ]:
import sys

# 2. Add your project path to Python's system path
# This tells Python: "Look in this folder when I try to import things"
project_path = '/content/drive/My Drive/CIS 5200 Final Project/code'

if project_path not in sys.path:
    sys.path.append(project_path)

# can use importlib to safely reload if you change the script
import flight_delay_pipeline
import importlib
importlib.reload(flight_delay_pipeline)
from flight_delay_pipeline import get_preprocessing_pipeline


print("Successfully loaded pipeline!")

Successfully loaded pipeline!


# Load and Prep Data (as necessary)

In [6]:
# i=1
# mod = f'{i}'
mod = "added_delay_20251202_034422"

DATA_FILE_PATH = "/content/drive/My Drive/CIS 5200 Final Project/train_test_datasets/"
checkpoint_dir = "/content/drive/My Drive/CIS 5200 Final Project/models/random_forests/"

# 1. Load the dataset and drop rows with missing values
X_train_path = DATA_FILE_PATH + f'X_train_{mod}.csv'
y_train_path = DATA_FILE_PATH + f'y_train_{mod}.csv'

X_test_path = DATA_FILE_PATH + f'X_test_{mod}.csv'
y_test_path = DATA_FILE_PATH + f'y_test_{mod}.csv'

# Regression
REGR_TARGET = 'DEP_ADDED_DELAY'

# Classification
CLASS_TARGET='DEP_DEL15'

## HyperParameter Randomized Search, cuML random forests

In [ ]:
# target = 'DEP_ADDED_DELAY'
# # 1. Load a SUBSET of data for tuning (e.g., 100k rows)
# # Tuning on the full 1.2M dataset is overkill and slow for CV
# X_tune = pd.read_csv(X_train_path, nrows=200000)
# y_tune = pd.read_csv(y_train_path, nrows=200000)[target]

# # 2. Get your existing preprocessing pipeline (without the model)
# # note: flight_delay_pipeline.py returns a pipeline ending in 'preprocessor'
# prep_pipeline = get_preprocessing_pipeline()

# # 3. Create the Full Pipeline (Prep + GPU Model)
# # We add the model as the final step
# full_pipeline = Pipeline(steps=[
#     ('prep', prep_pipeline),
#     ('model', cuRF_Regressor(verbose=0)) # Turn off model verbosity to keep output clean
# ])

# # 4. Define the Parameter Grid
# # Note the 'model__' prefix which tells sklearn to apply these to the 'model' step
# param_dist = {
#     'model__n_estimators': [100, 150, 200, 250, 300],     # More trees = better but slower
#     'model__max_depth': [10, 15, 20, 25, 30],       # Depth controls overfitting
#     'model__max_features': ['sqrt', 'log2'],    # Features per split
#     'model__n_bins': [32, 64, 128, 256],             # Lower bins = Faster, less GPU RAM
#     'model__min_samples_split': [2, 5, 10],
# }

# # 5. Setup RandomizedSearchCV
# search = RandomizedSearchCV(
#     estimator=full_pipeline,
#     param_distributions=param_dist,
#     n_iter=20,          # Try 20 random combinations
#     cv=3,               # 3-Fold CV is enough for tuning
#     scoring='neg_mean_absolute_error',
#     n_jobs=1,           # CRITICAL: Keep this at 1 to avoid GPU OOM
#     verbose=3,          # See progress
#     random_state=42
# )

# # 6. Run the search
# print("Starting Hyperparameter Sweep...")
# search.fit(X_tune, y_tune)

# print(f"Best Params: {search.best_params_}")
# # The "best_score_" will now be negative MAE (e.g., -15.4)
# print(f"Best MAE (Validation): {search.best_score_}")

# # 7. (Optional) Save the best params to use in your main script
# best_model_params = {k.replace('model__', ''): v for k, v in search.best_params_.items()}
# print("Params ready for train_model:", best_model_params)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

target = 'DEP_ADDED_DELAY'

# 1. Load a SUBSET of data for tuning (e.g., 100k rows)
X_tune = pd.read_csv(X_train_path, nrows=200000)
y_tune = pd.read_csv(y_train_path, nrows=200000)[target]

# 2. Get your existing preprocessing pipeline (without the model)
prep_pipeline = get_preprocessing_pipeline()

# 3. Create the Full Pipeline (Prep + RF Model)
full_pipeline = Pipeline(steps=[
    ('prep', prep_pipeline),
    ('model', RandomForestRegressor(
        n_jobs=-1,          # use all cores; set to 1 if you hit RAM issues
        random_state=42,
        oob_score=False
    )),
])

# 4. Define the Parameter Search Space
# Note the 'model__' prefix to target the RF inside the pipeline
param_dist = {
    'model__n_estimators':      [100, 200, 300, 500],       # more trees = slower but usually better
    'model__max_depth':         [10, 15, 20, 30],     # None = expand fully
    'model__min_samples_split': [2, 5, 10],                 # larger => more regularization
    'model__min_samples_leaf':  [1, 2, 4],                  # avoid leaves with tiny samples
    'model__max_features':      ['sqrt', 'log2', 0.3, 0.5], # features per split
    'model__bootstrap':         [True, False],              # with/without bootstrap sampling
    'model__criterion': [
        'squared_error',        # standard MSE
        'friedman_mse',         # can be slightly better for some problems
        # 'absolute_error',       # MAE; more robust, slower
        'poisson',              # for non-negative targets (fits delays well)
    ],
}

# 5. Setup RandomizedSearchCV
search = RandomizedSearchCV(
    estimator=full_pipeline,
    param_distributions=param_dist,
    n_iter=10,                      # number of random combos to try
    cv=2,                           # 3-fold CV
    scoring='neg_mean_absolute_error',
    n_jobs=-1,                      # parallelize across cores
    verbose=2,
    random_state=42,
)

# 6. Run the search
print("Starting RandomForest hyperparameter sweep...")
search.fit(X_tune, y_tune)

print("Best Params:", search.best_params_)
print("Best MAE (validation):", -search.best_score_)

# 7. (Optional) Strip the 'model__' prefix to plug into your main training script
best_model_params = {k.replace('model__', ''): v
                     for k, v in search.best_params_.items()}
print("Params ready for train_model():", best_model_params)


Starting RandomForest hyperparameter sweep...
Fitting 2 folds for each of 10 candidates, totalling 20 fits


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGABRT(-6)}
Detailed tracebacks of the workers should have been printed to stderr in the executor process if faulthandler was not disabled.

In [ ]:
import pprint

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

target = 'DEP_ADDED_DELAY'

# 1. Load a SUBSET of data for tuning (e.g., 100k rows)
X_tune = pd.read_csv(X_train_path, nrows=200000)
y_tune = pd.read_csv(y_train_path, nrows=200000)[target]

# 2. Get your existing preprocessing pipeline (without the model)
prep_pipeline = get_preprocessing_pipeline()

# 3. Create the Full Pipeline (Prep + RF Model)
full_pipeline = Pipeline(steps=[
    ('prep', prep_pipeline),
    ('model', RandomForestRegressor(
        n_jobs=-1,          # use all cores; set to 1 if you hit RAM issues
        random_state=42,
        oob_score=False
    )),
])

# 4. Define the Parameter Search Space
# Note the 'model__' prefix to target the RF inside the pipeline
param_dist = {
    'model__n_estimators':      [100, 200, 300, 500],       # more trees = slower but usually better
    'model__max_depth':         [None, 10, 15, 20, 30],     # None = expand fully
    'model__min_samples_split': [2, 5, 10],                 # larger => more regularization
    'model__min_samples_leaf':  [1, 2, 4],                  # avoid leaves with tiny samples
    'model__max_features':      ['sqrt', 'log2', 0.3, 0.5], # features per split
    'model__bootstrap':         [True, False],              # with/without bootstrap sampling
    'model__criterion': [
        'squared_error',        # standard MSE
        'friedman_mse',         # can be slightly better for some problems
        'absolute_error',       # MAE; more robust, slower
        'poisson',              # for non-negative targets (fits delays well)
    ],
}

# 5. Setup RandomizedSearchCV
search = RandomizedSearchCV(
    estimator=full_pipeline,
    param_distributions=param_dist,
    n_iter=40,                      # number of random combos to try
    cv=3,                           # 3-fold CV
    scoring='neg_mean_absolute_error',
    n_jobs=-1,                      # parallelize across cores
    verbose=2,
    random_state=42,
)

# 6. Run the search
print("Starting RandomForest hyperparameter sweep...")
search.fit(X_tune, y_tune)

print("Best Params:", search.best_params_)
print("Best MAE (validation):", -search.best_score_)

# 7. (Optional) Strip the 'model__' prefix to plug into your main training script
best_model_params = {k.replace('model__', ''): v
                     for k, v in search.best_params_.items()}
print("Params ready for train_model():", best_model_params)


Starting RandomForest hyperparameter sweep...
Fitting 3 folds for each of 40 candidates, totalling 120 fits


/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

target = 'DEP_ADDED_DELAY'

# 1. Load a SUBSET of data for tuning (e.g., 100k rows)
X_tune = pd.read_csv(X_train_path, nrows=50000)
y_tune = pd.read_csv(y_train_path, nrows=50000)[target]

# 2. Get your existing preprocessing pipeline (without the model)
prep_pipeline = get_preprocessing_pipeline()

# 3. Create the Full Pipeline (Prep + RF Model)
full_pipeline = Pipeline(steps=[
    ('prep', prep_pipeline),
    ('model', RandomForestRegressor(
        n_jobs=-1,          # use all cores; set to 1 if you hit RAM issues
        random_state=42,
        oob_score=False
    )),
])

# 4. Define the Parameter Search Space
# Note the 'model__' prefix to target the RF inside the pipeline
param_dist = {
    'model__n_estimators':      [100, 200, 300],       # more trees = slower but usually better
    'model__max_depth':         [10, 15, 20],     # None = expand fully
    'model__min_samples_split': [2, 5, 10],                 # larger => more regularization
    'model__min_samples_leaf':  [1, 2, 4],                  # avoid leaves with tiny samples
    'model__max_features':      ['sqrt', 'log2', 0.3, 0.5], # features per split
    'model__bootstrap':         [True, False],              # with/without bootstrap sampling
    'model__criterion': [
        'squared_error',        # standard MSE
        # 'friedman_mse',         # can be slightly better for some problems
        # 'absolute_error',       # MAE; more robust, slower
        'poisson',              # for non-negative targets (fits delays well)
    ],
}

# param_dist = {
#     'model__n_estimators':      [80, 120, 160],        # fewer options, fewer trees
#     'model__max_depth':         [10, 15, 20],          # no very deep trees
#     'model__min_samples_split': [2, 5],                # regularization
#     'model__min_samples_leaf':  [1, 2],                # avoid tiny leaves
#     'model__max_features':      ['sqrt', 0.3],         # limit features per split
#     'model__bootstrap':         [True],                # keep it simple
#     'model__criterion': [
#         'squared_error',        # standard MSE
#         'poisson',              # good for non-negative, skewed targets
#     ],
# }

# 5. Setup RandomizedSearchCV
search = RandomizedSearchCV(
    estimator=full_pipeline,
    param_distributions=param_dist,
    n_iter=20,                      # number of random combos to try
    cv=3,                           # 3-fold CV
    scoring='neg_mean_absolute_error',
    n_jobs=-1,                      # parallelize across cores
    verbose=2,
    random_state=42,
)

# 6. Run the search
print("Starting RandomForest hyperparameter sweep...")
search.fit(X_tune, y_tune)

print("Best Params:", search.best_params_)
print("Best MAE (validation):", -search.best_score_)

# 7. (Optional) Strip the 'model__' prefix to plug into your main training script
best_model_params = {k.replace('model__', ''): v
                     for k, v in search.best_params_.items()}
print("Params ready for train_model():", best_model_params)


Starting RandomForest hyperparameter sweep...
Fitting 2 folds for each of 5 candidates, totalling 10 fits
[cuml.accel] `RandomForestRegressor.fit` ran on GPU
Best Params: {'model__n_estimators': 120, 'model__min_samples_split': 5, 'model__min_samples_leaf': 2, 'model__max_features': 'sqrt', 'model__max_depth': 10, 'model__criterion': 'poisson', 'model__bootstrap': True}
Best MAE (validation): 8.781764030456543
Params ready for train_model(): {'n_estimators': 120, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'poisson', 'bootstrap': True}


In [ ]:

print('Best Parameters for RF with Randomized Grid Search')
pprint(best_model_params)

Best Parameters for RF with Randomized Grid Search


NameError: name 'pprint' is not defined

# Setup the Random Forest
Some important hyperparameters for a Random Forest:
1. n_estimators: the # of decision trees to ensemble (default=100)

2. max_features: the max # of features a decision tree will be allowed to consider (default = sqrt(N_features))

3. max_depth: max depth a tree will be allowed to reach (default = inf)

4. max_leaf_nodes: max number of leaf nodes a tree can hold (default = inf)

5. max_sample: how many samples from the training set that a decision tree can see (default = entire training set)

6. min_samples_split: The threshold for which a node can be split (default = 2)

More hyperparams [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

In [ ]:
from pprint import pprint

In [ ]:
# define hyperparameters for RandomForestRegressor
# n_estimators=5000
# max_features="sqrt"
# max_depth=16 # cuML default is 16, sklearn default is inf
# max_leaves=-1 #inf
# max_samples=1.0 # all samples
# min_samples_split=2
# criterion="squared_error" # "absolute error" # is an option, but much slower per the sklearn docs
# verbose=2
# n_binsint (default = 128) # Maximum number of bins used by the split algorithm per feature.
# For large problems, particularly those with highly-skewed input data, increasing the number of bins may improve accuracy.
# https://docs.rapids.ai/api/cuml/stable/api/#output-data-type-configuration, default is 'input', so for pandas inputs == numpy array outputs
best_model_params = {'n_estimators': 200, 'n_bins': 128, 'min_samples_split': 5, 'max_features': 'log2', 'max_depth': 15}


## Regression
# params = {'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 30,
#           'max_leaves': -1, 'max_samples': 1.0, 'min_samples_split': 2,
#           'n_bins': 256, 'criterion': 'squared_error', 'verbose': 2}

# params.update(best_model_params)
# pprint(params)
# model=cuRF_Regressor(**params)

## Classification
params = {'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 30,
          'max_leaves': -1, 'max_samples': 1.0, 'min_samples_split': 2,
          'n_bins': 256, 'criterion': 'entropy', 'verbose': 2}

params.update(best_model_params)
model=cuRF_Classifier(**params)

In [ ]:
pprint(X_train_path)
pprint(X_test_path)
pprint(y_train_path)
pprint(y_test_path)

('/content/drive/My Drive/CIS 5200 Final '
 'Project/train_test_datasets/X_train_added_delay_20251202_034422.csv')
('/content/drive/My Drive/CIS 5200 Final '
 'Project/train_test_datasets/X_test_added_delay_20251202_034422.csv')
('/content/drive/My Drive/CIS 5200 Final '
 'Project/train_test_datasets/y_train_added_delay_20251202_034422.csv')
('/content/drive/My Drive/CIS 5200 Final '
 'Project/train_test_datasets/y_test_added_delay_20251202_034422.csv')


# Train Model on Datasets

## Full Dataset

In [ ]:
# best_model_params = {'n_estimators': 200, 'n_bins': 128, 'min_samples_split': 5, 'max_features': 'log2', 'max_depth': 15}


# Regression
params = {'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 30,
          'max_leaves': -1, 'max_samples': 1.0, 'min_samples_split': 2,
          'n_bins': 256, 'criterion': 'squared_error', 'verbose': 2}

params.update(best_model_params)
pprint(params)
model=cuRF_Regressor(**params)

## Classification
# params = {'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 30,
#           'max_leaves': -1, 'max_samples': 1.0, 'min_samples_split': 2,
#           'n_bins': 256, 'criterion': 'entropy', 'verbose': 2}

# params.update(best_model_params)
# pprint(params)
# model=cuRF_Classifier(**params)

NameError: name 'best_model_params' is not defined

In [7]:
results_reg = train_regressor(
    X_train_path,
    y_train_path,
    X_test_path,
    y_test_path,
    description="RF_Reg_pos_only",
    pos_only=True,
    checkpoint_dir=checkpoint_dir
)


--- Starting Regressor Training: RF_Reg_pos_only ---
  Loading Training Data...
  Filtering for positive delays... (Original: 5086383)
  Training samples remaining: 877028
  Creating Validation Split (15%)...
  Preprocessing & Fitting RandomForestRegressor...
[cuml.accel] `RandomForestRegressor.fit` ran on GPU
  Training finished in 35.0s
  Saving model to: /content/drive/My Drive/CIS 5200 Final Project/models/random_forests/RF_Reg_pos_only_20251207_195407.joblib ...
[cuml.accel] `RandomForestRegressor` fitted attributes synced to CPU
  Evaluating on Test set examples...
  pos_only is True, testing on only positive examples
[cuml.accel] `RandomForestRegressor.predict` ran on GPU
  RMSE:  21.9396
  MAE:   17.4261
  MAPE:  234.9407%
  Metrics saved to: /content/drive/My Drive/CIS 5200 Final Project/models/random_forests/RF_Reg_pos_only_20251207_195407_metrics.json
